In [1]:
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf


Diabetes data set

https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset

In [2]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

In [3]:
X.shape

(442, 10)

In [4]:
y.shape

(442,)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression With SKLearn

In [9]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression().fit(X_train, y_train)

In [17]:
from sklearn.metrics import mean_squared_error
import math

y_validation_hat = lr_model.predict(X_validation)
val_rmse = math.sqrt(mean_squared_error(y_validation, y_validation_hat))
y_train_hat = lr_model.predict(X_train)
train_rmse = math.sqrt(mean_squared_error(y_train, y_train_hat))


print("RMSE on training data: {}".format(train_rmse))
print("RMSE on validation data: {}".format(val_rmse))

RMSE on training data: 53.55881425327268
RMSE on validation data: 53.8532569849144


# Gradient Boost Tree with Xgboost

In [26]:
from xgboost import XGBRegressor

my_model = XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.1,
    max_depth=3,
    gamma=10,
    min_child_weight=10,
    n_estimators=200, 
#     subsample=0.87,
    colsample_bynode=1,
#     colsample_bytree=0.9,
    reg_alpha=0,
    reg_lambda=3,
    
    booster='gbtree',
    tree_method='hist',
    missing=None,
    verbosity=1
)

print(my_model.get_xgb_params())

trained = my_model.fit(
    X_train, y_train, early_stopping_rounds=10,
    # the last metric is used for early stopping
    eval_metric=["rmse"],
    eval_set=[(X_train, y_train), (X_validation, y_validation)], verbose=True
)


{'objective': 'reg:squarederror', 'base_score': None, 'booster': 'gbtree', 'colsample_bylevel': None, 'colsample_bynode': 1, 'colsample_bytree': None, 'gamma': 10, 'gpu_id': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_delta_step': None, 'max_depth': 3, 'min_child_weight': 10, 'monotone_constraints': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 0, 'reg_lambda': 3, 'scale_pos_weight': None, 'subsample': None, 'tree_method': 'hist', 'validate_parameters': None, 'verbosity': 1}
[0]	validation_0-rmse:157.33365	validation_1-rmse:148.61986
[1]	validation_0-rmse:144.35011	validation_1-rmse:136.29663
[2]	validation_0-rmse:132.74573	validation_1-rmse:125.11211
[3]	validation_0-rmse:122.43217	validation_1-rmse:115.12044
[4]	validation_0-rmse:113.20927	validation_1-rmse:106.47181
[5]	validation_0-rmse:104.99681	validation_1-rmse:98.47568
[6]	validation_0-rmse:97.72016	validation_1-rmse:91.62320
[7]	validation_0-rmse:91.27415	validation_1

# Linear Regression In Neural Network Style

In [54]:
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Reshape, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import MeanSquaredError



In [59]:
def build_linear_regression():
    inputs = Input(shape=(10,))
    l = inputs    
    l = Dense(1, activation="linear", name='fc')(l)
    linear_regression_model = Model(inputs=inputs, outputs=l)
    print(linear_regression_model.summary())
    return linear_regression_model

def model_train(X_train, y_train, X_val, y_val, model):
    print(model.summary())
    early_stop = EarlyStopping(
        monitor="val_loss", min_delta=0, patience=3,
        verbose=0, mode="auto", baseline=None, restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, min_lr=0.05)

    callbacks_list = [early_stop, reduce_lr]

    # optimizer: gradient descent implementation
    adam_wn = Adam(learning_rate=0.8)
    
    # compile model with loss and optimizer; loss function needs to match the intention and the output of the last layer
    model.compile(
        loss="mean_squared_error", optimizer=adam_wn,
        metrics=[MeanSquaredError()]
    )
    
    # kick off model training
    return model.fit(
        x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=442,
        epochs=10000, verbose=1, shuffle=True,
        callbacks=callbacks_list)


In [60]:

my_lr_model = build_linear_regression()

model_train(X_train, y_train, X_validation, y_validation, my_lr_model)

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 10)]              0         
_________________________________________________________________
fc (Dense)                   (None, 1)                 11        
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________
None
Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 10)]              0         
_________________________________________________________________
fc (Dense)                   (None, 1)                 11        
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10000
1/1 [===================

Epoch 40/10000
1/1 [==============================] - 0s 16ms/step - loss: 20521.9121 - mean_squared_error: 20521.9121 - val_loss: 17973.5977 - val_mean_squared_error: 17973.5977
Epoch 41/10000
1/1 [==============================] - 0s 16ms/step - loss: 20321.6172 - mean_squared_error: 20321.6172 - val_loss: 17793.9219 - val_mean_squared_error: 17793.9219
Epoch 42/10000
1/1 [==============================] - 0s 21ms/step - loss: 20123.1152 - mean_squared_error: 20123.1152 - val_loss: 17615.9160 - val_mean_squared_error: 17615.9160
Epoch 43/10000
1/1 [==============================] - 0s 17ms/step - loss: 19926.3926 - mean_squared_error: 19926.3926 - val_loss: 17439.5762 - val_mean_squared_error: 17439.5762
Epoch 44/10000
1/1 [==============================] - 0s 17ms/step - loss: 19731.4453 - mean_squared_error: 19731.4453 - val_loss: 17264.8926 - val_mean_squared_error: 17264.8926
Epoch 45/10000
1/1 [==============================] - 0s 17ms/step - loss: 19538.2715 - mean_squared_erro

Epoch 86/10000
1/1 [==============================] - 0s 17ms/step - loss: 13040.5977 - mean_squared_error: 13040.5977 - val_loss: 11316.4336 - val_mean_squared_error: 11316.4336
Epoch 87/10000
1/1 [==============================] - 0s 17ms/step - loss: 12913.9248 - mean_squared_error: 12913.9248 - val_loss: 11204.9346 - val_mean_squared_error: 11204.9346
Epoch 88/10000
1/1 [==============================] - 0s 16ms/step - loss: 12788.6191 - mean_squared_error: 12788.6191 - val_loss: 11094.6943 - val_mean_squared_error: 11094.6943
Epoch 89/10000
1/1 [==============================] - 0s 16ms/step - loss: 12664.6738 - mean_squared_error: 12664.6738 - val_loss: 10985.7031 - val_mean_squared_error: 10985.7031
Epoch 90/10000
1/1 [==============================] - 0s 16ms/step - loss: 12542.0762 - mean_squared_error: 12542.0762 - val_loss: 10877.9512 - val_mean_squared_error: 10877.9512
Epoch 91/10000
1/1 [==============================] - 0s 16ms/step - loss: 12420.8145 - mean_squared_erro

1/1 [==============================] - 0s 18ms/step - loss: 8477.4424 - mean_squared_error: 8477.4424 - val_loss: 7344.3418 - val_mean_squared_error: 7344.3418
Epoch 133/10000
1/1 [==============================] - 0s 17ms/step - loss: 8403.4316 - mean_squared_error: 8403.4316 - val_loss: 7280.9043 - val_mean_squared_error: 7280.9043
Epoch 134/10000
1/1 [==============================] - 0s 17ms/step - loss: 8330.3418 - mean_squared_error: 8330.3418 - val_loss: 7218.2969 - val_mean_squared_error: 7218.2969
Epoch 135/10000
1/1 [==============================] - 0s 16ms/step - loss: 8258.1660 - mean_squared_error: 8258.1660 - val_loss: 7156.5156 - val_mean_squared_error: 7156.5156
Epoch 136/10000
1/1 [==============================] - 0s 17ms/step - loss: 8186.8945 - mean_squared_error: 8186.8945 - val_loss: 7095.5493 - val_mean_squared_error: 7095.5493
Epoch 137/10000
1/1 [==============================] - 0s 17ms/step - loss: 8116.5200 - mean_squared_error: 8116.5200 - val_loss: 7035.3

Epoch 179/10000
1/1 [==============================] - 0s 16ms/step - loss: 5860.8687 - mean_squared_error: 5860.8687 - val_loss: 5134.2397 - val_mean_squared_error: 5134.2397
Epoch 180/10000
1/1 [==============================] - 0s 16ms/step - loss: 5821.3979 - mean_squared_error: 5821.3979 - val_loss: 5101.5806 - val_mean_squared_error: 5101.5806
Epoch 181/10000
1/1 [==============================] - 0s 16ms/step - loss: 5782.4839 - mean_squared_error: 5782.4839 - val_loss: 5069.4106 - val_mean_squared_error: 5069.4106
Epoch 182/10000
1/1 [==============================] - 0s 15ms/step - loss: 5744.1196 - mean_squared_error: 5744.1196 - val_loss: 5037.7217 - val_mean_squared_error: 5037.7217
Epoch 183/10000
1/1 [==============================] - 0s 16ms/step - loss: 5706.2988 - mean_squared_error: 5706.2988 - val_loss: 5006.5088 - val_mean_squared_error: 5006.5088
Epoch 184/10000
1/1 [==============================] - 0s 16ms/step - loss: 5669.0161 - mean_squared_error: 5669.0161 - 

...

1/1 [==============================] - 0s 16ms/step - loss: 2922.4597 - mean_squared_error: 2922.4597 - val_loss: 2902.2891 - val_mean_squared_error: 2902.2891
Epoch 1246/10000
1/1 [==============================] - 0s 16ms/step - loss: 2922.3904 - mean_squared_error: 2922.3904 - val_loss: 2902.2644 - val_mean_squared_error: 2902.2644
Epoch 1247/10000
1/1 [==============================] - 0s 16ms/step - loss: 2922.3215 - mean_squared_error: 2922.3215 - val_loss: 2902.2397 - val_mean_squared_error: 2902.2397
Epoch 1248/10000
1/1 [==============================] - 0s 16ms/step - loss: 2922.2529 - mean_squared_error: 2922.2529 - val_loss: 2902.2151 - val_mean_squared_error: 2902.2151
Epoch 1249/10000
1/1 [==============================] - 0s 16ms/step - loss: 2922.1843 - mean_squared_error: 2922.1843 - val_loss: 2902.1909 - val_mean_squared_error: 2902.1909
Epoch 1250/10000
1/1 [==============================] - 0s 16ms/step - loss: 2922.1157 - mean_squared_error: 2922.1157 - val_loss: 2

1/1 [==============================] - 0s 16ms/step - loss: 2919.4602 - mean_squared_error: 2919.4602 - val_loss: 2901.3269 - val_mean_squared_error: 2901.3269
Epoch 1292/10000
1/1 [==============================] - 0s 16ms/step - loss: 2919.3992 - mean_squared_error: 2919.3992 - val_loss: 2901.3098 - val_mean_squared_error: 2901.3098
Epoch 1293/10000
1/1 [==============================] - 0s 17ms/step - loss: 2919.3379 - mean_squared_error: 2919.3379 - val_loss: 2901.2930 - val_mean_squared_error: 2901.2930
Epoch 1294/10000
1/1 [==============================] - 0s 16ms/step - loss: 2919.2769 - mean_squared_error: 2919.2769 - val_loss: 2901.2764 - val_mean_squared_error: 2901.2764
Epoch 1295/10000
1/1 [==============================] - 0s 16ms/step - loss: 2919.2161 - mean_squared_error: 2919.2161 - val_loss: 2901.2598 - val_mean_squared_error: 2901.2598
Epoch 1296/10000
1/1 [==============================] - 0s 16ms/step - loss: 2919.1558 - mean_squared_error: 2919.1558 - val_loss: 2

1/1 [==============================] - 0s 16ms/step - loss: 2916.8052 - mean_squared_error: 2916.8052 - val_loss: 2900.6917 - val_mean_squared_error: 2900.6917
Epoch 1338/10000
1/1 [==============================] - 0s 16ms/step - loss: 2916.7512 - mean_squared_error: 2916.7512 - val_loss: 2900.6812 - val_mean_squared_error: 2900.6812
Epoch 1339/10000
1/1 [==============================] - 0s 16ms/step - loss: 2916.6968 - mean_squared_error: 2916.6968 - val_loss: 2900.6704 - val_mean_squared_error: 2900.6704
Epoch 1340/10000
1/1 [==============================] - 0s 16ms/step - loss: 2916.6431 - mean_squared_error: 2916.6431 - val_loss: 2900.6599 - val_mean_squared_error: 2900.6599
Epoch 1341/10000
1/1 [==============================] - 0s 16ms/step - loss: 2916.5894 - mean_squared_error: 2916.5894 - val_loss: 2900.6499 - val_mean_squared_error: 2900.6499
Epoch 1342/10000
1/1 [==============================] - 0s 15ms/step - loss: 2916.5352 - mean_squared_error: 2916.5352 - val_loss: 2

1/1 [==============================] - 0s 17ms/step - loss: 2914.4500 - mean_squared_error: 2914.4500 - val_loss: 2900.3169 - val_mean_squared_error: 2900.3169
Epoch 1384/10000
1/1 [==============================] - 0s 16ms/step - loss: 2914.4021 - mean_squared_error: 2914.4021 - val_loss: 2900.3113 - val_mean_squared_error: 2900.3113
Epoch 1385/10000
1/1 [==============================] - 0s 16ms/step - loss: 2914.3540 - mean_squared_error: 2914.3540 - val_loss: 2900.3059 - val_mean_squared_error: 2900.3059
Epoch 1386/10000
1/1 [==============================] - 0s 16ms/step - loss: 2914.3059 - mean_squared_error: 2914.3059 - val_loss: 2900.3003 - val_mean_squared_error: 2900.3003
Epoch 1387/10000
1/1 [==============================] - 0s 16ms/step - loss: 2914.2578 - mean_squared_error: 2914.2578 - val_loss: 2900.2947 - val_mean_squared_error: 2900.2947
Epoch 1388/10000
1/1 [==============================] - 0s 15ms/step - loss: 2914.2104 - mean_squared_error: 2914.2104 - val_loss: 2

1/1 [==============================] - 0s 16ms/step - loss: 2912.3555 - mean_squared_error: 2912.3555 - val_loss: 2900.1472 - val_mean_squared_error: 2900.1472
Epoch 1430/10000
1/1 [==============================] - 0s 15ms/step - loss: 2912.3130 - mean_squared_error: 2912.3130 - val_loss: 2900.1455 - val_mean_squared_error: 2900.1455
Epoch 1431/10000
1/1 [==============================] - 0s 16ms/step - loss: 2912.2695 - mean_squared_error: 2912.2695 - val_loss: 2900.1438 - val_mean_squared_error: 2900.1438
Epoch 1432/10000
1/1 [==============================] - 0s 16ms/step - loss: 2912.2271 - mean_squared_error: 2912.2271 - val_loss: 2900.1421 - val_mean_squared_error: 2900.1421
Epoch 1433/10000
1/1 [==============================] - 0s 16ms/step - loss: 2912.1843 - mean_squared_error: 2912.1843 - val_loss: 2900.1404 - val_mean_squared_error: 2900.1404
Epoch 1434/10000
1/1 [==============================] - 0s 20ms/step - loss: 2912.1421 - mean_squared_error: 2912.1421 - val_loss: 2